In [ ]:
!pip install -U --pre tensorflow=="2.4"
!pip install tf_slim

In [ ]:
!pip install pycocotools

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%%bash 
cd models/research
pip install .

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import imageio
from datetime import datetime
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [ ]:
import cv2

cap = cv2.VideoCapture(0)   # if you have multiple webcams change the value to the correct one

In [ ]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
#/content/models/research/object_detection/data
#PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90


In [ ]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    #od_graph_def = tf.GraphDef()
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
!pwd

/content


In [ ]:
#/content/models/research/object_detection/data
#PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
%cd models/research/object_detection/

/content/models/research/object_detection


In [ ]:
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [ ]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]  # change this value if you want to add more pictures to test

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [ ]:
!pwd
#upload video


/content/models/research/object_detection


In [ ]:
tf.executing_eagerly()

True

In [ ]:
with detection_graph.as_default():



  #tf.compat.v1.Session
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
    # Each box represents a part of the image where a particular object was detected.
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
    input_video='pp' #name of video
    video_reader=imageio.get_reader('%s.mp4'%input_video)
    print(video_reader)
    video_writer=imageio.get_writer('%s_output.mp4'%input_video,fps=10)

    t0=datetime.now()
    n_frame=0
    for frame in video_reader:
      image_np=frame
      frame+=1
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      print(type(image_np_expanded))
      #p=boxes , q=scores, r=classes, s=num_detection
      (p, q, r, s) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: np.expand_dims(image_np, 0)})
        #  feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(p),
          np.squeeze(r).astype(np.int32),
          np.squeeze(q),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
      for i,b in enumerate(p[0]):
        print('p1')
        #                 car                    bus                  truck
        if r[0][i] == 3 or r[0][i] == 6 or r[0][i] == 8:
          print("come in p1 1")

          if q[0][i] >= 0.5:
            mid_x = (p[0][i][1]+p[0][i][3])/2
            mid_y = (p[0][i][0]+p[0][i][2])/2
            apx_distance = round(((1 - (p[0][i][3] - p[0][i][1]))**4),1)
            cv2.putText(image_np, '{}'.format(apx_distance), (int(mid_x*800),int(mid_y*450)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

            if apx_distance <=0.5:
              if mid_x > 0.3 and mid_x < 0.7:
                #cv2.putText(image_np, 'WARNING!!!', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,0,0), 3)
                cv2.putText(image_np, 'COLLISION WARNING!', (int(mid_x*800)-50, int(mid_y*450)), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,0,0), 3)
      video_writer.append_data(image_np)    
    fps=n_frame/(datetime.now()-t0).total_seconds()
    print('frame process %s, speed %s fps'%(n_frame,fps))
    video_writer.close()

Streaming output truncated to the last 5000 lines.
p1
p1
come in p1 1
p1
come in p1 1
p1
p1
come in p1 1
p1
p1
come in p1 1
p1
come in p1 1
p1
p1
p1
p1
p1
p1
p1
come in p1 1
p1
p1
p1
p1
come in p1 1
p1
p1
p1
p1
p1
p1
p1
p1
come in p1 1
p1
p1
p1
p1
p1
p1
p1
come in p1 1
p1
p1
come in p1 1
p1
come in p1 1
p1
come in p1 1
p1
p1
come in p1 1
<class 'imageio.core.util.Array'>
p1
come in p1 1
p1
come in p1 1
p1
come in p1 1
p1
come in p1 1
p1
p1
come in p1 1
p1
come in p1 1
p1
come in p1 1
p1
p1
p1
come in p1 1
p1
come in p1 1
p1
come in p1 1
p1
come in p1 1
p1
p1
p1
p1
come in p1 1
p1
come in p1 1
p1
p1
come in p1 1
p1
p1
come in p1 1
p1
p1
p1
p1
p1
come in p1 1
p1
p1
p1
p1
come in p1 1
p1
p1
come in p1 1
p1
come in p1 1
p1
p1
p1
p1
p1
p1
p1
come in p1 1
p1
come in p1 1
p1
p1
p1
p1
p1
p1
come in p1 1
p1
p1
p1
come in p1 1
p1
come in p1 1
p1
p1
p1
p1
p1
p1
p1
p1
come in p1 1
p1
p1
p1
p1
come in p1 1
p1
p1
p1
p1
p1
come in p1 1
p1
come in p1 1
p1
come in p1 1
p1
come in p1 1
p1
p1
come in p1 

In [ ]:
for f in range(0,3):
  print(f)
  